In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

In [ ]:
# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.

seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 40, 40, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))


In [11]:
seq.summary()
#seq.compile(loss='binary_crossentropy', optimizer='adadelta')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 40, 40, 40)  59200     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 40, 40, 40)  160       
_________________________________________________________________
conv_lst_m2d_4 (ConvLSTM2D)  (None, None, 40, 40, 40)  115360    
__________

In [1]:
def process_data(n_samples,data,n_frames=10,rows=40,cols=40,channels=1):
    x=[]
    y=[]
    for i in range(n_samples - n_frames):
        row=[]
        for j in range(n_frames):
            row.append(list(data[i+j]))
            
        y.append(data[i+n_frames]) 
        row=np.array(row)
        row=row.flatten()
        x.append(row)
    x=np.array(x)
    y=np.array(y)
    
    x=np.reshape(x,(n_samples- n_frames,n_frames,rows,cols,channels))
    y=np.reshape(y,(n_samples- n_frames, 1 ,rows, cols, channels))
    
    return x,y
    

In [13]:
import numpy as np
data=np.loadtxt('test_40.csv',delimiter='\t',dtype='float32')

print(data.shape)

(1000, 1600)


In [14]:
x,y=process_data(n_samples=1000,data=data)

print(x.shape)
print(y.shape)

(990, 10, 40, 40, 1)
(990, 1, 40, 40, 1)


In [ ]:
seq.fit(x, y, batch_size=30,shuffle=False,
        epochs=50, validation_split=0.05)

Train on 940 samples, validate on 50 samples
Epoch 1/50


In [ ]:
seq.save('convlstm_model.h5')

In [4]:
from keras.models import load_model
seq = load_model('convlstm_model.h5')

Using TensorFlow backend.


In [8]:
predicted=seq.predict(x)

In [9]:
predicted.shape

(990, 10, 40, 40, 1)

In [10]:
predicted=predicted.reshape(990,1600)
predicted.savetxt('predicted_40.csv',delimiter='\t')

ValueError: cannot reshape array of size 15840000 into shape (990,1600)

In [ ]:
# Testing the network on one movie
# feed it with the first 7 positions and then
# predict the new positions
which = 989
track = x[which][:7, ::, ::, ::]

for j in range(11):
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::])
    new = new_pos[::, -1, ::, ::, ::]
    track = np.concatenate((track, new), axis=0)

In [ ]:
# And then compare the predictions
# to the ground truth
import matplotlib.pyplot as plt
%matplotlib inline 

track2 = x[which][::, ::, ::, ::]
for i in range(50):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(121)

    if i >= 7:
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20)

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot)
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = x[which][i - 1, ::, ::, 0]

    plt.imshow(toplot)
    plt.savefig('output_images/%i_animate.png' % (i + 1))